<a href="https://colab.research.google.com/github/DinRazar/Unesco_parse/blob/main/Unesco_parser_xml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import xml.etree.ElementTree as ET
import json
import re

# Страны для фильтрации
target_countries = ['ru', 'by', 'ua', 'lt', 'lv', 'ee']

def parse_xml_to_geojson(xml_file_path, output_geojson_path):
    # Парсим XML
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Создаем структуру GeoJSON
    geojson = {
        "type": "FeatureCollection",
        "features": []
    }

    # Проходим по всем строкам
    for row in root.findall('row'):
        # Получаем страны объекта из iso_code
        iso_code_elem = row.find('iso_code')
        if iso_code_elem is None or iso_code_elem.text is None:
            continue

        iso_codes = iso_code_elem.text.split(',')

        # Проверяем, есть ли среди стран наши целевые
        if not any(country in iso_codes for country in target_countries):
            continue

        # Извлекаем информацию
        site_name = row.find('site').text if row.find('site') is not None else ""
        category = row.find('category').text if row.find('category') is not None else ""
        year = row.find('date_inscribed').text if row.find('date_inscribed') is not None else ""
        states = row.find('states').text if row.find('states') is not None else ""

        # Извлекаем геолокации
        geolocations = row.find('geolocations')
        if geolocations is None:
            continue

        # Для каждого POI создаем отдельную фичу
        for poi in geolocations.findall('poi'):
            iso2 = poi.find('iso2').text if poi.find('iso2') is not None else ""

            # Проверяем, что POI относится к целевой стране
            if iso2 not in target_countries:
                continue

            lat_elem = poi.find('latitude')
            lon_elem = poi.find('longitude')

            if lat_elem is None or lon_elem is None or lat_elem.text is None or lon_elem.text is None:
                continue

            try:
                lat = float(lat_elem.text)
                lon = float(lon_elem.text)
            except ValueError:
                continue

            # Создаем фичу
            feature = {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [lon, lat]
                },
                "properties": {
                    "site": site_name,
                    "category": category,
                    "year": year,
                    "country": iso2.upper(),
                    "states": states,
                    "iso_code": iso_codes,
                    "latitude": lat,
                    "longitude": lon
                }
            }

            geojson["features"].append(feature)

    # Сохраняем в файл
    with open(output_geojson_path, 'w', encoding='utf-8') as f:
        json.dump(geojson, f, ensure_ascii=False, indent=2)

    print(f"Найдено {len(geojson['features'])} объектов")
    print(f"Результат сохранен в: {output_geojson_path}")

    # Статистика по странам
    country_stats = {}
    for feature in geojson["features"]:
        country = feature["properties"]["country"]
        country_stats[country] = country_stats.get(country, 0) + 1

    print("\nСтатистика по странам:")
    for country, count in country_stats.items():
        print(f"  {country}: {count} объектов")

def create_simplified_dataset(xml_file_path, output_csv_path):
    """Создает упрощенный CSV датасет"""
    import csv

    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['site', 'category', 'year', 'country', 'states', 'latitude', 'longitude']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()

        for row in root.findall('row'):
            iso_code_elem = row.find('iso_code')
            if iso_code_elem is None or iso_code_elem.text is None:
                continue

            iso_codes = iso_code_elem.text.split(',')

            if not any(country in iso_codes for country in target_countries):
                continue

            site_name = row.find('site').text if row.find('site') is not None else ""
            category = row.find('category').text if row.find('category') is not None else ""
            year = row.find('date_inscribed').text if row.find('date_inscribed') is not None else ""
            states = row.find('states').text if row.find('states') is not None else ""

            geolocations = row.find('geolocations')
            if geolocations is None:
                continue

            for poi in geolocations.findall('poi'):
                iso2 = poi.find('iso2').text if poi.find('iso2') is not None else ""

                if iso2 not in target_countries:
                    continue

                lat_elem = poi.find('latitude')
                lon_elem = poi.find('longitude')

                if lat_elem is None or lon_elem is None or lat_elem.text is None or lon_elem.text is None:
                    continue

                try:
                    lat = float(lat_elem.text)
                    lon = float(lon_elem.text)
                except ValueError:
                    continue

                writer.writerow({
                    'site': site_name,
                    'category': category,
                    'year': year,
                    'country': iso2.upper(),
                    'states': states,
                    'latitude': lat,
                    'longitude': lon
                })

    print(f"\nCSV датасет сохранен в: {output_csv_path}")

# Основная часть программы
if __name__ == "__main__":
    xml_file = "/content/xml.xml"  # Ваш XML файл
    geojson_output = "unesco_objects_baltics.geojson"
    csv_output = "unesco_objects_baltics.csv"

    # Создаем GeoJSON
    parse_xml_to_geojson(xml_file, geojson_output)

    # Создаем CSV для удобного анализа
    create_simplified_dataset(xml_file, csv_output)

    # Дополнительная информация
    print("\n" + "="*50)
    print("ДАННЫЕ ГОТОВЫ К ИСПОЛЬЗОВАНИЮ")
    print("="*50)
    print("\nФорматы данных:")
    print("1. GeoJSON - для визуализации на картах")
    print("2. CSV - для анализа в Excel/Pandas")
    print("\nПример использования GeoJSON в Python:")
    print("""
import geopandas as gpd
import matplotlib.pyplot as plt

# Загрузка данных
gdf = gpd.read_file('unesco_objects_baltics.geojson')

# Визуализация
fig, ax = plt.subplots(figsize=(12, 8))
gdf.plot(column='country', legend=True, ax=ax, markersize=50)
plt.title('Объекты ЮНЕСКО в странах Балтии и соседних')
plt.show()
""")

Найдено 262 объектов
Результат сохранен в: unesco_objects_baltics.geojson

Статистика по странам:
  UA: 42 объектов
  BY: 8 объектов
  LT: 7 объектов
  LV: 4 объектов
  EE: 4 объектов
  RU: 197 объектов

CSV датасет сохранен в: unesco_objects_baltics.csv

ДАННЫЕ ГОТОВЫ К ИСПОЛЬЗОВАНИЮ

Форматы данных:
1. GeoJSON - для визуализации на картах
2. CSV - для анализа в Excel/Pandas

Пример использования GeoJSON в Python:

import geopandas as gpd
import matplotlib.pyplot as plt

# Загрузка данных
gdf = gpd.read_file('unesco_objects_baltics.geojson')

# Визуализация
fig, ax = plt.subplots(figsize=(12, 8))
gdf.plot(column='country', legend=True, ax=ax, markersize=50)
plt.title('Объекты ЮНЕСКО в странах Балтии и соседних')
plt.show()



In [4]:
!pip install folium geopandas pandas requests pillow -q

In [14]:
import folium
import geopandas as gpd
import pandas as pd
import json
import requests
from io import BytesIO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from folium import IFrame
import base64

# Загрузка GeoJSON данных
print("Загрузка данных...")
with open('unesco_objects_baltics.geojson', 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

# Преобразуем в GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])

# Словарь для цвета стран
country_colors = {
    'RU': 'red',     # Россия
    'BY': 'green',   # Беларусь
    'UA': 'blue',    # Украина
    'LT': 'yellow',  # Литва
    'LV': 'orange',  # Латвия
    'EE': 'purple'   # Эстония
}

# Словарь для полных названий стран
country_names = {
    'RU': 'Россия',
    'BY': 'Беларусь',
    'UA': 'Украина',
    'LT': 'Литва',
    'LV': 'Латвия',
    'EE': 'Эстония'
}

# Скачиваем флаги стран
print("Загрузка флагов стран...")
flags = {}
flag_urls = {
    'RU': 'https://flagcdn.com/w40/ru.png',
    'BY': 'https://flagcdn.com/w40/by.png',
    'UA': 'https://flagcdn.com/w40/ua.png',
    'LT': 'https://flagcdn.com/w40/lt.png',
    'LV': 'https://flagcdn.com/w40/lv.png',
    'EE': 'https://flagcdn.com/w40/ee.png'
}

# for country_code, url in flag_urls.items():
#     try:
#         response = requests.get(url)
#         flags[country_code] = response.content
#         print(f"  ✓ Флаг {country_names[country_code]} загружен")
#     except Exception as e:
#         print(f"  ✗ Ошибка загрузки флага {country_names[country_code]}: {e}")

# Упрощенная функция для создания круглого маркера с флагом
def create_flag_icon(country_code):
    if country_code in flags:
        try:
            # Создаем круглый флаг
            img = Image.open(BytesIO(flags[country_code]))

            # Создаем круглую маску
            mask = Image.new('L', img.size, 0)
            draw = ImageDraw.Draw(mask)
            draw.ellipse([(0, 0), img.size], fill=255)

            # Применяем маску
            img.putalpha(mask)

            # Сохраняем в буфер
            buffered = BytesIO()
            img.save(buffered, format="PNG")
            img_str = base64.b64encode(buffered.getvalue()).decode()

            # Создаем HTML для иконки
            html = f'''
            <div style="
                border-radius: 50%;
                overflow: hidden;
                width: 40px;
                height: 40px;
                border: 2px solid white;
                box-shadow: 0 2px 4px rgba(0,0,0,0.3);
            ">
                <img src="data:image/png;base64,{img_str}"
                     style="width: 100%; height: 100%; object-fit: cover;">
            </div>
            '''
            return folium.DivIcon(html=html)
        except Exception as e:
            print(f"  Ошибка обработки флага {country_code}: {e}")
            # Используем цветной кружок как запасной вариант

    # Альтернатива: цветной кружок с буквой страны
    return folium.Icon(
        color=country_colors.get(country_code, 'gray'),
        icon='circle',
        icon_color='white',
        prefix='fa',
        html=f'<div style="font-size: 12px; font-weight: bold;">{country_code}</div>'
    )

# Функция для создания поп-апа с информацией
def create_popup_html(row):
    html = f"""
    <div style="min-width: 250px; font-family: Arial, sans-serif;">
        <h4 style="margin-top: 0; color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 5px;">
            {row['site']}
        </h4>
        <p><strong>📍 Страна:</strong> <span style="color: {country_colors.get(row['country'], '#000')};
                        font-weight: bold;">{country_names.get(row['country'], row['country'])}</span></p>
        <p><strong>📋 Категория:</strong> {row['category']}</p>
        <p><strong>📅 Год внесения:</strong> <span style="background-color: #f0f0f0; padding: 2px 6px;
                        border-radius: 3px;">{row['year']}</span></p>
        <p><strong>🌍 Страны-участники:</strong> <br>{row['states']}</p>
        <p><strong>🗺️ Координаты:</strong><br>
           <span style="font-family: monospace;">Широта: {row['latitude']:.4f}<br>Долгота: {row['longitude']:.4f}</span></p>
    </div>
    """
    return html

# Создаем базовую карту (центрируем на регионе)
print("\nСоздание карты...")
center_lat = gdf['latitude'].mean()
center_lon = gdf['longitude'].mean()

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=4,
    tiles='CartoDB positron',
    control_scale=True
)

# Добавляем слой кластеризации
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

# Добавляем маркеры на карту
print("Добавление маркеров на карту...")
for idx, row in gdf.iterrows():
    icon = create_flag_icon(row['country'])
    popup = folium.Popup(create_popup_html(row), max_width=300)

    # Создаем маркер
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=popup,
        icon=icon,
        tooltip=f"📍 {row['site']} ({country_names.get(row['country'], row['country'])})"
    ).add_to(marker_cluster)

# Создаем FeatureGroup для каждой страны
for country_code in country_colors.keys():
    country_data = gdf[gdf['country'] == country_code]
    if len(country_data) > 0:
        fg = folium.FeatureGroup(
            name=f"🇷🇺 {country_names[country_code]} ({len(country_data)})",
            overlay=True
        )

        # Добавляем маркеры в FeatureGroup
        for idx, row in country_data.iterrows():
            icon = folium.Icon(
                color=country_colors.get(country_code, 'gray'),
                icon='circle',
                icon_color='white',
                prefix='fa'
            )

            popup = folium.Popup(create_popup_html(row), max_width=300)

            folium.Marker(
                location=[row['latitude'], row['longitude']],
                popup=popup,
                icon=icon
            ).add_to(fg)

        fg.add_to(m)

# Добавляем контроль слоев
folium.LayerControl().add_to(m)

# Добавляем красивую легенду
legend_html = '''
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 220px;
    height: auto;
    background-color: white;
    border: 2px solid #3498db;
    border-radius: 8px;
    z-index: 9999;
    font-family: Arial, sans-serif;
    font-size: 14px;
    padding: 15px;
    box-shadow: 0 4px 12px rgba(0,0,0,0.15);
">
    <h4 style="
        margin-top: 0;
        margin-bottom: 15px;
        color: #2c3e50;
        border-bottom: 1px solid #eee;
        padding-bottom: 8px;
    ">
        🗺️ Объекты ЮНЕСКО
    </h4>
'''

for code, name in country_names.items():
    color = country_colors[code]
    count = len(gdf[gdf['country'] == code])

    # Для стран с флагами показываем флаг, для остальных - цветной кружок
    if code in flags:
        try:
            img = Image.open(BytesIO(flags[code])).resize((20, 20))
            buffered = BytesIO()
            img.save(buffered, format="PNG")
            img_str = base64.b64encode(buffered.getvalue()).decode()

            legend_html += f'''
            <div style="display: flex; align-items: center; margin: 8px 0;">
                <img src="data:image/png;base64,{img_str}"
                     style="width: 20px; height: 20px; border-radius: 50%; border: 1px solid #ccc; margin-right: 10px;">
                <span>{name}: <strong>{count}</strong></span>
            </div>
            '''
        except:
            # Если не удалось обработать флаг, используем цветной кружок
            legend_html += f'''
            <div style="display: flex; align-items: center; margin: 8px 0;">
                <div style="background-color: {color}; width: 20px; height: 20px;
                            border-radius: 50%; margin-right: 10px; border: 1px solid #ccc;"></div>
                <span>{name}: <strong>{count}</strong></span>
            </div>
            '''
    else:
        legend_html += f'''
        <div style="display: flex; align-items: center; margin: 8px 0;">
            <div style="background-color: {color}; width: 20px; height: 20px;
                        border-radius: 50%; margin-right: 10px; border: 1px solid #ccc;"></div>
            <span>{name}: <strong>{count}</strong></span>
        </div>
        '''

legend_html += '''
    <hr style="margin: 15px 0; border: none; border-top: 1px solid #eee;">
    <div style="text-align: center; color: #7f8c8d; font-size: 12px;">
        Всего объектов: <strong>{}</strong>
    </div>
</div>
'''.format(len(gdf))

m.get_root().html.add_child(folium.Element(legend_html))

# Добавляем заголовок
title_html = '''
<div style="
    position: fixed;
    top: 10px;
    left: 50px;
    right: 50px;
    z-index: 1000;
    background-color: rgba(255, 255, 255, 0.95);
    padding: 12px 20px;
    border-radius: 8px;
    border: 2px solid #3498db;
    font-family: Arial, sans-serif;
    box-shadow: 0 4px 12px rgba(0,0,0,0.1);
">
    <h2 style="margin: 0; color: #2c3e50; font-size: 20px;">
        🏛️ Объекты Всемирного наследия ЮНЕСКО
    </h2>
    <p style="margin: 5px 0 0 0; color: #7f8c8d; font-size: 14px;">
        Восточная Европа и страны Балтии
    </p>
</div>
'''

m.get_root().html.add_child(folium.Element(title_html))

# # Добавляем кнопки управления
# controls_html = '''
# <div style="
#     position: fixed;
#     top: 100px;
#     right: 20px;
#     z-index: 1000;
#     background-color: white;
#     border-radius: 8px;
#     border: 1px solid #ddd;
#     padding: 10px;
#     box-shadow: 0 2px 6px rgba(0,0,0,0.1);
# ">
#     <div style="font-weight: bold; margin-bottom: 5px;">Быстрые действия:</div>
#     <button onclick="document.querySelector('.leaflet-control-layers').click()"
#             style="background: #3498db; color: white; border: none; padding: 5px 10px;
#                    border-radius: 4px; cursor: pointer; margin: 2px;">
#         Слои карты
#     </button>
#     <button onclick="window.open('unesco_map.html', '_blank')"
#             style="background: #27ae60; color: white; border: none; padding: 5px 10px;
#                    border-radius: 4px; cursor: pointer; margin: 2px;">
#         Сохранить карту
#     </button>
# </div>
# '''

# m.get_root().html.add_child(folium.Element(controls_html))

# # Показываем статистику
# print(f"\n📊 СТАТИСТИКА ОБЪЕКТОВ:")
# print("=" * 40)
# for code, name in country_names.items():
#     count = len(gdf[gdf['country'] == code])
#     print(f"  🇷🇺 {name}: {count} объектов")

# print(f"\nВсего объектов: {len(gdf)}")
# print(f"\n📈 Распределение по категориям:")
# for category in gdf['category'].unique():
#     count = len(gdf[gdf['category'] == category])
#     print(f"  {category}: {count} объектов")

# Отображаем карту
print("\n" + "=" * 60)
print("✅ Карта успешно создана! Отображаю...")
print("=" * 60 + "\n")
m

Загрузка данных...
Загрузка флагов стран...

Создание карты...
Добавление маркеров на карту...

✅ Карта успешно создана! Отображаю...



/tmp/ipython-input-1434502405.py:101: UserWarning: color argument of Icon should be one of: {'lightblue', 'white', 'purple', 'beige', 'darkblue', 'blue', 'cadetblue', 'darkgreen', 'lightred', 'darkpurple', 'orange', 'lightgray', 'pink', 'darkred', 'green', 'black', 'gray', 'red', 'lightgreen'}.
  return folium.Icon(
/tmp/ipython-input-1434502405.py:169: UserWarning: color argument of Icon should be one of: {'lightblue', 'white', 'purple', 'beige', 'darkblue', 'blue', 'cadetblue', 'darkgreen', 'lightred', 'darkpurple', 'orange', 'lightgray', 'pink', 'darkred', 'green', 'black', 'gray', 'red', 'lightgreen'}.
  icon = folium.Icon(
